In [1]:
from utilities import fetch_competitions, request_headers, Historize, commit_changes, createDetailedURL, polite_request
from sql_shortcuts import table, append_table, load_keys, drop_table, tables, replace_table
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import sqlite3
import hashlib
import requests
import time
import random
import logging

### Leagues

In [2]:
# Load retrieved league data
logging.info("FETCHING LEAGUES")
leagues_history = table("b1_league")

# Scrape new league data
league_data = fetch_competitions()
leagues =  pd.DataFrame(league_data, columns =  ['name', 'league_link'])
leagues = pd.merge(leagues.assign(joined =  1), leagues_history, on =  ['name', 'league_link'], how = 'outer')
leagues = leagues[leagues.joined != 1]

#Update b1_league
if len(leagues > 0):
    logging.info(f"Committing {len(leagues)} league records")
    leagues = leagues.drop("joined", axis = 1)
    append_table(leagues, league)
else:
    logging.info("League table up-to-date")

2025-06-02 19:22:58,581 - INFO - FETCHING LEAGUES
2025-06-02 19:23:00,324 - INFO - League table up-to-date


### Clubs

In [3]:
logging.info("FETCHING CLUBS")

leagues = table("b1_league")
league_urls = list(leagues.league_link.unique())
club_df = pd.DataFrame()
b2_failures = []


#Scrape clubs
for url in league_urls:

    response = polite_request(url)
    if not isinstance(response, str):
        data = pd.read_html(StringIO(response.text))
        df = [i for i in data if 'total market value' in str(i).lower()][0]
        df["league_link"] = url
        club_df = pd.concat([club_df, df])
    else:
        b2_failures.append(url)

# deduplicate, clean columns
club_df = club_df.reset_index(drop = True)
club_df = club_df.dropna(subset = 'Club.1')
selection = ['Club.1', 'league_link']
club_df = club_df[selection].drop_duplicates().rename({"Club.1" : "club"}, axis = 1)

if len(b2_failures)>0:
    logging.error(f"b2_club failures: {b2_failures}")

2025-06-02 19:23:00,363 - INFO - FETCHING CLUBS
2025-06-02 19:23:07,476 - INFO - Successfully retrieved https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
2025-06-02 19:23:13,138 - INFO - Successfully retrieved https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1
2025-06-02 19:23:19,708 - INFO - Successfully retrieved https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1
2025-06-02 19:23:26,771 - INFO - Successfully retrieved https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1
2025-06-02 19:23:32,564 - INFO - Successfully retrieved https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1
2025-06-02 19:23:38,634 - INFO - Successfully retrieved https://www.transfermarkt.com/liga-portugal/startseite/wettbewerb/PO1
2025-06-02 19:23:44,567 - INFO - Successfully retrieved https://www.transfermarkt.com/eredivisie/startseite/wettbewerb/NL1
2025-06-02 19:23:51,139 - INFO - Successfully retrieved https://www.transfermarkt.com/jupiler-p

In [4]:
processor = Historize(
    club_df,
    primary_key = ['league_link', 'club']
)

processor.run()

club_updates = processor.df

club_updates['last_updated'] = datetime.today()
club_updates = club_updates.rename({"ID": "club_id"}, axis = 1)
selection = ["club_id", 'club', 'league_link', 'hash_key', 'effective_start_date', 'effective_end_date', 'last_updated']
club_updates = club_updates[selection] 

commit_changes(
    df = club_updates, 
    table_name = 'b2_club', 
    primary_key= 'club_id'
)

2025-06-02 19:28:53,630 - INFO - Adding 0 records to b2_club
2025-06-02 19:28:53,632 - INFO - 0 records being updated - effective_end_date == 2025-06-01



Continue? (y/n)
y


2025-06-02 19:32:36,786 - INFO - Update and append complete (b2_club)


### Club Links

In [5]:
logging.info("FETCHING CLUB LINKS")
league_urls = table('b2_club')['league_link'].unique()
club_links = []

for idx, league_url in enumerate(league_urls):
        
    response = requests.get(league_url, headers=request_headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table containing the teams
    temp = soup.select('table.items tbody tr')
    for row in soup.select('table.items tbody tr'):
        link_tag = row.select_one('td.hauptlink a')
        if link_tag:
            club_name = link_tag.text.strip()
            relative_link = link_tag['href']
            #filter out top scorer links
            if r"profil/spieler" not in relative_link:
                full_link = "https://www.transfermarkt.com" + relative_link
                club_links.append((league_url, club_name, full_link))
            

2025-06-02 19:32:36,810 - INFO - FETCHING CLUB LINKS


In [6]:
# deduplicate / filter links
club_links_df = pd.DataFrame(club_links, columns = ['league_link', 'club', 'club_link'])
club_links_df = club_links_df[club_links_df.club_link.str.contains("start")] 

In [7]:
processor = Historize(
    club_links_df,
    primary_key = ['league_link', 'club']
)

processor.run()
updates = processor.df
updates['last_updated'] = datetime.today()
updates = updates.rename({"ID": "club_link_id"}, axis = 1)
selection = ['club_link_id', 'league_link', 'club', 'club_link', 'hash_key', 'effective_start_date', 'effective_end_date', 'last_updated']
updates = updates[selection]

In [12]:
type(datetime.today())

datetime.datetime

datetime.datetime

In [14]:
temp = table('b3_club_link')
temp['last_updated'] = datetime.now() - timedelta(2)

In [15]:
replace_table('b3_club_link', temp)

Are you sure you want to overwrite this table? This will overwrite the existing data permanently (y/n)
y


In [16]:
commit_changes(
    df = updates, 
    table_name = 'b3_club_link', 
    primary_key='club_link_id'
)

2025-06-03 07:40:34,471 - INFO - Adding 12 records to b3_club_link
2025-06-03 07:40:34,475 - INFO - 12 records being updated - effective_end_date == 2025-06-02



Continue? (y/n)
y


2025-06-03 07:40:36,112 - INFO - Update and append complete (b3_club_link)


### Players

In [17]:
logging.info("FETCHING PLAYERS")
club_links = table("b3_club_link")
club_links["club_link_detailed"] = club_links['club_link'].apply(lambda x: createDetailedURL(x))
urls = club_links.club_link_detailed.unique()

start_time = time.time()

conn = sqlite3.connect("transfermarkt.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS b4_player_staging")
conn.commit()
conn.close()

b4_failures = []
for idx, url in enumerate(urls):
    
    response = polite_request(url)

    if not isinstance(response, str):
        data = pd.read_html(StringIO(response.text))
        players = [i for i in data if 'Contract' in i.columns][0].dropna(subset = "#").copy()
        players['club_link_detailed'] = url
        logging.info(f"Adding {len(players)} to staging: {url}")
        append_table(players, "b4_player_staging")
        
    else:
        b4_failures.append(url)
        
logging.info(f"{round((time.time() - start_time)/60)} minutes to process")

if len(b4_failures)>0:
    logging.error(f"b4_player failures: {b4_failures}")

2025-06-03 07:40:39,188 - INFO - FETCHING PLAYERS
2025-06-03 07:41:01,950 - INFO - Successfully retrieved https://www.transfermarkt.com/manchester-city/kader/verein/281/saison_id/2024/plus/1
2025-06-03 07:41:02,111 - INFO - Adding 28 to staging: https://www.transfermarkt.com/manchester-city/kader/verein/281/saison_id/2024/plus/1
2025-06-03 07:41:08,463 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-arsenal/kader/verein/11/saison_id/2024/plus/1
2025-06-03 07:41:08,594 - INFO - Adding 24 to staging: https://www.transfermarkt.com/fc-arsenal/kader/verein/11/saison_id/2024/plus/1
2025-06-03 07:41:13,133 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-liverpool/kader/verein/31/saison_id/2024/plus/1
2025-06-03 07:41:13,251 - INFO - Adding 24 to staging: https://www.transfermarkt.com/fc-liverpool/kader/verein/31/saison_id/2024/plus/1
2025-06-03 07:41:19,898 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-chelsea/kader/verein/631/saison_id/202

2025-06-03 07:51:12,549 - INFO - Successfully retrieved https://www.transfermarkt.com/real-betis-sevilla/kader/verein/150/saison_id/2024/plus/1 on attempt 2
2025-06-03 07:51:12,650 - INFO - Adding 25 to staging: https://www.transfermarkt.com/real-betis-sevilla/kader/verein/150/saison_id/2024/plus/1
2025-06-03 07:51:36,716 - INFO - Successfully retrieved https://www.transfermarkt.com/ud-las-palmas/kader/verein/472/saison_id/2024/plus/1 on attempt 2
2025-06-03 07:51:36,828 - INFO - Adding 25 to staging: https://www.transfermarkt.com/ud-las-palmas/kader/verein/472/saison_id/2024/plus/1
2025-06-03 07:52:02,113 - INFO - Successfully retrieved https://www.transfermarkt.com/ca-osasuna/kader/verein/331/saison_id/2024/plus/1 on attempt 2
2025-06-03 07:52:02,200 - INFO - Adding 22 to staging: https://www.transfermarkt.com/ca-osasuna/kader/verein/331/saison_id/2024/plus/1
2025-06-03 07:52:50,446 - INFO - Successfully retrieved https://www.transfermarkt.com/celta-vigo/kader/verein/940/saison_id/20

2025-06-03 08:04:50,650 - INFO - Adding 29 to staging: https://www.transfermarkt.com/holstein-kiel/kader/verein/269/saison_id/2024/plus/1
2025-06-03 08:05:14,387 - INFO - Successfully retrieved https://www.transfermarkt.com/inter-mailand/kader/verein/46/saison_id/2024/plus/1 on attempt 2
2025-06-03 08:05:14,491 - INFO - Adding 26 to staging: https://www.transfermarkt.com/inter-mailand/kader/verein/46/saison_id/2024/plus/1
2025-06-03 08:05:19,767 - INFO - Successfully retrieved https://www.transfermarkt.com/juventus-turin/kader/verein/506/saison_id/2024/plus/1
2025-06-03 08:05:19,917 - INFO - Adding 27 to staging: https://www.transfermarkt.com/juventus-turin/kader/verein/506/saison_id/2024/plus/1
2025-06-03 08:06:49,020 - INFO - Successfully retrieved https://www.transfermarkt.com/ac-mailand/kader/verein/5/saison_id/2024/plus/1 on attempt 4
2025-06-03 08:06:49,124 - INFO - Adding 26 to staging: https://www.transfermarkt.com/ac-mailand/kader/verein/5/saison_id/2024/plus/1
2025-06-03 08:0

2025-06-03 08:14:20,865 - INFO - Successfully retrieved https://www.transfermarkt.com/stade-brest-29/kader/verein/3911/saison_id/2024/plus/1
2025-06-03 08:14:20,993 - INFO - Adding 27 to staging: https://www.transfermarkt.com/stade-brest-29/kader/verein/3911/saison_id/2024/plus/1
2025-06-03 08:14:48,060 - INFO - Successfully retrieved https://www.transfermarkt.com/stade-reims/kader/verein/1421/saison_id/2024/plus/1 on attempt 2
2025-06-03 08:14:48,166 - INFO - Adding 26 to staging: https://www.transfermarkt.com/stade-reims/kader/verein/1421/saison_id/2024/plus/1
2025-06-03 08:15:35,802 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-nantes/kader/verein/995/saison_id/2024/plus/1 on attempt 3
2025-06-03 08:15:35,911 - INFO - Adding 26 to staging: https://www.transfermarkt.com/fc-nantes/kader/verein/995/saison_id/2024/plus/1
2025-06-03 08:15:44,876 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-toulouse/kader/verein/415/saison_id/2024/plus/1
2025-06-03 0

2025-06-03 08:24:25,647 - INFO - Adding 29 to staging: https://www.transfermarkt.com/ajax-amsterdam/kader/verein/610/saison_id/2024/plus/1
2025-06-03 08:24:33,027 - INFO - Successfully retrieved https://www.transfermarkt.com/az-alkmaar/kader/verein/1090/saison_id/2024/plus/1
2025-06-03 08:24:33,139 - INFO - Adding 30 to staging: https://www.transfermarkt.com/az-alkmaar/kader/verein/1090/saison_id/2024/plus/1
2025-06-03 08:24:38,802 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-twente-enschede/kader/verein/317/saison_id/2024/plus/1
2025-06-03 08:24:38,899 - INFO - Adding 26 to staging: https://www.transfermarkt.com/fc-twente-enschede/kader/verein/317/saison_id/2024/plus/1
2025-06-03 08:25:27,872 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-utrecht/kader/verein/200/saison_id/2024/plus/1 on attempt 3
2025-06-03 08:25:27,969 - INFO - Adding 27 to staging: https://www.transfermarkt.com/fc-utrecht/kader/verein/200/saison_id/2024/plus/1
2025-06-03 08:25:

...

2025-06-03 09:18:56,081 - INFO - Adding 24 to staging: https://www.transfermarkt.com/austin-fc/kader/verein/72309/saison_id/2024/plus/1
2025-06-03 09:19:06,661 - INFO - Successfully retrieved https://www.transfermarkt.com/columbus-crew-sc/kader/verein/813/saison_id/2024/plus/1
2025-06-03 09:19:06,810 - INFO - Adding 26 to staging: https://www.transfermarkt.com/columbus-crew-sc/kader/verein/813/saison_id/2024/plus/1
2025-06-03 09:19:13,742 - INFO - Successfully retrieved https://www.transfermarkt.com/charlotte-fc/kader/verein/78435/saison_id/2024/plus/1
2025-06-03 09:19:13,910 - INFO - Adding 32 to staging: https://www.transfermarkt.com/charlotte-fc/kader/verein/78435/saison_id/2024/plus/1
2025-06-03 09:19:21,514 - INFO - Successfully retrieved https://www.transfermarkt.com/vancouver-whitecaps-fc/kader/verein/6321/saison_id/2024/plus/1
2025-06-03 09:19:21,659 - INFO - Adding 25 to staging: https://www.transfermarkt.com/vancouver-whitecaps-fc/kader/verein/6321/saison_id/2024/plus/1
2025-

2025-06-03 09:29:18,022 - INFO - Successfully retrieved https://www.transfermarkt.com/clube-atletico-mineiro/kader/verein/330/saison_id/2024/plus/1 on attempt 2
2025-06-03 09:29:18,149 - INFO - Adding 30 to staging: https://www.transfermarkt.com/clube-atletico-mineiro/kader/verein/330/saison_id/2024/plus/1
2025-06-03 09:30:48,885 - INFO - Successfully retrieved https://www.transfermarkt.com/fc-santos/kader/verein/221/saison_id/2024/plus/1 on attempt 4
2025-06-03 09:30:49,025 - INFO - Adding 33 to staging: https://www.transfermarkt.com/fc-santos/kader/verein/221/saison_id/2024/plus/1
2025-06-03 09:32:25,617 - ERROR - Failed to fetch https://www.transfermarkt.com/esporte-clube-bahia/kader/verein/10010/saison_id/2024/plus/1 on 4 retries. Status code: 503
2025-06-03 09:33:55,942 - INFO - Successfully retrieved https://www.transfermarkt.com/ec-cruzeiro-belo-horizonte/kader/verein/609/saison_id/2024/plus/1 on attempt 4
2025-06-03 09:33:56,084 - INFO - Adding 30 to staging: https://www.transf

2025-06-03 09:43:17,407 - INFO - Adding 27 to staging: https://www.transfermarkt.com/atlas-guadalajara/kader/verein/8590/saison_id/2024/plus/1
2025-06-03 09:43:22,360 - INFO - Successfully retrieved https://www.transfermarkt.com/puebla-fc/kader/verein/5662/saison_id/2024/plus/1
2025-06-03 09:43:22,501 - INFO - Adding 27 to staging: https://www.transfermarkt.com/puebla-fc/kader/verein/5662/saison_id/2024/plus/1
2025-06-03 09:43:39,807 - INFO - Successfully retrieved https://www.transfermarkt.com/queretaro-fc/kader/verein/4961/saison_id/2024/plus/1
2025-06-03 09:43:39,968 - INFO - Adding 31 to staging: https://www.transfermarkt.com/queretaro-fc/kader/verein/4961/saison_id/2024/plus/1
2025-06-03 09:43:48,241 - INFO - Successfully retrieved https://www.transfermarkt.com/mamelodi-sundowns-fc/kader/verein/6356/saison_id/2024/plus/1
2025-06-03 09:43:48,424 - INFO - Adding 40 to staging: https://www.transfermarkt.com/mamelodi-sundowns-fc/kader/verein/6356/saison_id/2024/plus/1
2025-06-03 09:43

2025-06-03 09:52:21,374 - INFO - Successfully retrieved https://www.transfermarkt.com/shams-azar-ghazvin/kader/verein/87232/saison_id/2024/plus/1 on attempt 2
2025-06-03 09:52:21,528 - INFO - Adding 35 to staging: https://www.transfermarkt.com/shams-azar-ghazvin/kader/verein/87232/saison_id/2024/plus/1
2025-06-03 09:52:55,271 - INFO - Successfully retrieved https://www.transfermarkt.com/chadormalu-sc/kader/verein/100931/saison_id/2024/plus/1 on attempt 2
2025-06-03 09:52:55,404 - INFO - Adding 31 to staging: https://www.transfermarkt.com/chadormalu-sc/kader/verein/100931/saison_id/2024/plus/1
2025-06-03 09:53:18,519 - INFO - Successfully retrieved https://www.transfermarkt.com/esteghlal-khuzestan/kader/verein/40472/saison_id/2024/plus/1 on attempt 2
2025-06-03 09:53:18,641 - INFO - Adding 30 to staging: https://www.transfermarkt.com/esteghlal-khuzestan/kader/verein/40472/saison_id/2024/plus/1
2025-06-03 09:53:31,293 - INFO - Successfully retrieved https://www.transfermarkt.com/mes-rafs

2025-06-03 10:00:01,246 - INFO - Successfully retrieved https://www.transfermarkt.com/al-shamal/kader/verein/10775/saison_id/2024/plus/1
2025-06-03 10:00:01,387 - INFO - Adding 26 to staging: https://www.transfermarkt.com/al-shamal/kader/verein/10775/saison_id/2024/plus/1
2025-06-03 10:01:30,965 - INFO - Successfully retrieved https://www.transfermarkt.com/qatar-sc/kader/verein/965/saison_id/2024/plus/1 on attempt 4
2025-06-03 10:01:31,100 - INFO - Adding 30 to staging: https://www.transfermarkt.com/qatar-sc/kader/verein/965/saison_id/2024/plus/1
2025-06-03 10:01:36,441 - INFO - Successfully retrieved https://www.transfermarkt.com/umm-salal-sc/kader/verein/10920/saison_id/2024/plus/1
2025-06-03 10:01:36,583 - INFO - Adding 25 to staging: https://www.transfermarkt.com/umm-salal-sc/kader/verein/10920/saison_id/2024/plus/1
2025-06-03 10:01:41,410 - INFO - Successfully retrieved https://www.transfermarkt.com/al-shahania-sc/kader/verein/34694/saison_id/2024/plus/1
2025-06-03 10:01:41,542 - 

In [19]:
#Staging cleaning
players = table("b4_player_staging")
players = players.dropna(subset = "#")
mapping = {'#':'number', 'Date of birth/Age':'age'}
players = players.rename(mapping, axis = 1)
players.columns = [i.lower().strip().replace(" ", "_") for i in players.columns]
players = players.drop(["nat.", "signed_from"], axis = 1)

In [20]:
player_historizer = Historize(
    players,
    primary_key = ['player', 'club_link_detailed']
)

In [21]:
player_historizer.run()
player_updates = player_historizer.df
player_updates = player_updates.rename({"ID": "player_id"}, axis = 1)
player_updates['last_updated'] = datetime.today()

In [ ]:
commit_changes(
    df = player_updates, 
    table_name = 'b4_player', 
    primary_key='player_id'
)

2025-06-03 10:32:33,392 - INFO - Adding 1355 records to b4_player
2025-06-03 10:32:33,395 - INFO - 1355 records being updated - effective_end_date == 2025-06-02



Continue? (y/n)
